# TIGRE benchmark visualization

In [1]:
import os
import sys
from pathlib import Path
from os import PathLike

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
data_dir = Path.cwd() / "benchmark/"    # expect this to be project_root/src/benchmark/
data_files = list(data_dir.glob("*.json"))
data_files[0]

WindowsPath('E:/gatech/courses/cse-6230-high-performance-parallel-computing/assignments/cse6230-final/src/benchmark/NumGPU_1_NameGPU_Tesla V100-PCIE-32GB_NumRound_5_Data_shepp_CGLS_only.json')

In [3]:
dfs = []
for f in data_files:
    df = pd.read_json(str(f))
    name_parts = f.stem.split("_")
    # print(name_parts)
    num_gpus = int(name_parts[name_parts.index("NumGPU") + 1])
    df['num_gpus'] = num_gpus
    dfs.append(df)
df: pd.DataFrame = pd.concat(dfs)
df.head()

,method,image_size,nangles,average_time,nRMSE,niter,num_gpus
0,CGLS,32,50,1.503667,102.003609,5.0,1
1,CGLS,32,100,1.517155,101.167366,5.0,1
2,CGLS,32,200,1.576894,101.124181,5.0,1
3,CGLS,32,400,1.640932,101.105221,5.0,1
4,CGLS,32,800,1.765042,101.107012,5.0,1


In [4]:
df = df.rename(
    columns={
        'method': 'Method',
        'image_size': 'Image Size',
        'nangles': 'Number of angles',
        'average_time': 'Average Time',
        'niter': 'Number of iterations',
        'num_gpus': 'Number of GPUs',
    }
)
df.head()

,Method,Image Size,Number of angles,Average Time,nRMSE,Number of iterations,Number of GPUs
0,CGLS,32,50,1.503667,102.003609,5.0,1
1,CGLS,32,100,1.517155,101.167366,5.0,1
2,CGLS,32,200,1.576894,101.124181,5.0,1
3,CGLS,32,400,1.640932,101.105221,5.0,1
4,CGLS,32,800,1.765042,101.107012,5.0,1


In [5]:
dofs = ['Image Size', 'Number of angles', 'Number of GPUs']
targets = ['nRMSE', 'Average Time']

figure_format = '.jpg'
figure_output_dir = Path.cwd() / 'figures'
if not figure_output_dir.exists():
    figure_output_dir.mkdir()

for d in dofs:
    print(f"DOF: {d}")
    config = list(set(dofs).difference({d}))
    groups = df.groupby(config)

    # plot for each
    for row_index, (key, item) in enumerate(groups):
        # print(item)
        # print((key, config))
        for col_index, t in enumerate(targets):
            # build figure name
            config_parts = []
            for config_name, config_value in zip(config, key):
                config_parts.append(f'{config_name}.{config_value}')
            figure_name = '-'.join(config_parts)
            figure_name = f'{figure_name}-{t}'

            # build figure title
            config_parts = []
            for config_name, config_value in zip(config, key):
                config_parts.append(f'{config_name}={config_value}')
            figure_title = ' '.join(config_parts)

            # plot the figure
            p = sns.lineplot(
                data=item,
                x=d,
                y=t,
                hue='Method',
            )
            p.set_title(figure_title)
            save_file = figure_output_dir / f'{figure_name}{figure_format}'
            print(f'Saving figure to {str(save_file)}')
            p.figure.savefig(save_file)
            p.figure.clf()

DOF: Image Size
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\Number of angles.50-Number of GPUs.1-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\Number of angles.50-Number of GPUs.1-Average Time.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\Number of angles.50-Number of GPUs.2-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\Number of angles.50-Number of GPUs.2-Average Time.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\Number of angles.50-Number of GPUs.3-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\Number of angles.50-Number 

<Figure size 432x288 with 0 Axes>

Plot FDK and Iterative methods on separate figures.

This is done because many attributes of FDK does not fall on the same scale as iterative methods (example: FDK has exponentially better runtime than iterative methods)

In [6]:
dofs = ['Image Size', 'Number of angles', 'Number of GPUs']
targets = ['nRMSE', 'Average Time']

figure_format = '.jpg'
figure_output_dir = Path.cwd() / 'figures/comparison'
if not figure_output_dir.exists():
    figure_output_dir.mkdir()

methods = df['Method'].unique()

for d in dofs:
    print(f"DOF: {d}")
    config = list(set(dofs).difference({d}))
    groups = df.groupby(config)

    # plot for each
    for _, (key, item) in enumerate(groups):
        # print(item)
        # print((key, config))
        for __, t in enumerate(targets):
            # build figure name <config_name>.<config_value>-<config_name>.<config_value>-...
            config_parts = []
            for config_name, config_value in zip(config, key):
                config_parts.append(f'{config_name}.{config_value}')
            figure_name = '-'.join(config_parts)
            figure_name = f'{figure_name}-{t}'


            # build figure title
            config_parts = []
            for config_name, config_value in zip(config, key):
                config_parts.append(f'{config_name}={config_value}')
            figure_title = ' '.join(config_parts)

            # n_groups x 2 columns, col=0 method == fdk, col=1 method=iterative
            figure, axes = plt.subplots(nrows=1, ncols=len(methods))
            # plot fdk on the left column and iterative methods on the right column
            for i, method_name in enumerate(methods):
                sns.lineplot(
                    data=item[item['Method'] == method_name],
                    x=d,
                    y=t,
                    hue='Method',
                    ax=axes[i]
                )
            axes[0].set_title(figure_title)

            save_file = figure_output_dir / f'{figure_name}{figure_format}'
            print(f'Saving figure to {str(save_file)}')
            figure.savefig(save_file)
            figure.clf()

DOF: Image Size
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.50-Number of GPUs.1-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.50-Number of GPUs.1-Average Time.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.50-Number of GPUs.2-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.50-Number of GPUs.2-Average Time.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.50-Number of GPUs.3-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignment

<ipython-input-6-73ea5507b4dd>:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure, axes = plt.subplots(nrows=1, ncols=len(methods))


Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.200-Number of GPUs.3-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.200-Number of GPUs.3-Average Time.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.200-Number of GPUs.4-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.200-Number of GPUs.4-Average Time.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-final\src\figures\comparison\Number of angles.400-Number of GPUs.1-nRMSE.jpg
Saving figure to E:\gatech\courses\cse-6230-high-performance-parallel-computing\assignments\cse6230-f

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>